In [3]:
:dep harrison_rust = {path = "."}

To run this notebook, you will need (in addition to Jupyter) a Jupyter Rust kernel, e.g. https://github.com/evcxr/evcxr/blob/main/evcxr_jupyter/README.md.

We will look at a semi-decision procedure for checking whether a first-order formula is valid.

/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////


Some quick starting definitions:

Let $\mathscr{L}$ be a language of first-order logic (which will usually be implied), and let $\phi(\bar{x})$ be an $\mathscr{L}$-formula with free variables $\bar{x}$.  (Note that $\bar{x}$ may be empty, in which case we say that $\phi$ is a *sentence*).

(**Validity**) We say that $\phi(\bar{x})$ is (first-order) *valid* if for *all* $\mathscr{L}$-models $M$ and *all* possible valuations $v$,  from $\bar{x}$ into elements of the domain $dom(M)$ we have $M,v \vDash \phi(\bar{x})$.

(**Satisfiability**) We say that $\phi(\bar{x})$ is (first-order) *satisfiable* if there exists *at least one* $\mathscr{L}$-model $M$ such that for *all* possible valuations $v$ from $\bar{x}$ into elements of the domain $dom(M)$ we have $M,v \vDash \phi(\bar{x})$

(Note that in both definitions we have a universal quantifier over valuations.  If you're just trying to say e.g. that "$\phi$ is possible for some tuple, use constants instead of variables, ie. "$\phi(\bar{c})$ is satisfiable".  For purposes of validity and satisfiability, it helpful to think of $\phi(\bar{x})$ as $\forall \bar{x} \phi(\bar{x})$.)

/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////


Our proceedure will make use of three major pieces of machinery:

1)  (Skolem) Every formula of first order logic is *equisatisfiable* to a quantifier free one.
2)  (Herbrand) A quantifier free formula of first-order logic is first-order satisfiable just in case the set of all its *ground instances* is *propositionally satisfiable*., and
3)  (Compactness) the (possibly infinite) set of formulas is satisfiable just in case every finite subset is satisfiable.

At a high level, our algorithm will convert our case formula to quantifier-free and check larger and larger sets of ground instances for unsatisfiability.  First we will walk through the Skolemization process.  Next we will look at the concepts involved in Herbrand's threorem, although we will not go through the proof.  (We take Compactness as given.)   Finally we look at the complete semi-decision proceedure.  Throughout we will use the `harrison_rust` package to provide concrete examples throughout.  You can find the source code here:

https://github.com/aetilley/harrison-rust

/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////


I)  **Skolemization**

(Every formula of first order logic is equisatisfiable to some quantifier free formula.)

Recall the definitions:

**Equisatisfiability**:  $\phi$ and $\psi$ are *equisatisfiable* just in case $\phi$ is satisfiable iff $\psi$ is.  Note that this is strictly weaker than their being *equivalent* (ie. being satisfied in the same interpretations).

**PNF**:  A formula $\phi$ is in Prenex Normal Form (PNF) if there is a quantifier-free formula $\psi$, integer $n$, and quantifiers ${Q_i: 1 \leq i \leq n}$ such that $\phi = Q_{n} x_{n}\ldots Q_{1} x_{1}\psi$

Theorem:  Every first-order formula has an equivalent formula in prenex normal form.

Our package has a tool for doing this conversion:

https://github.com/aetilley/harrison-rust/blob/main/src/first_order_logic.rs#L1633

In [3]:
use harrison_rust::formula::Formula;
use harrison_rust::first_order_logic::Pred;

let formula_string = "exists x. F(x, z) ==> forall z. ~G(z, x)";
let formula = Formula::<Pred>::parse(formula_string).unwrap();
formula.pprint();

<<(exists x. F(x, z)) ==> (forall z. ~G(z, x))>>


In PNF...

In [4]:
formula.pnf().pprint();

<<forall x' z'. (~F(x', z) \/ ~G(z', x))>>


So for our Skolemization task, it suffices to focus on ways to remove leading quantifiers.

Universal quantifiers are easy, since the semantics of formulas with free variables is such that  $\forall \bar{x} \phi(\bar{x})$ and $\phi(\bar{x})$ are equisatisfiable.

What about $\exists \bar{x} \phi(\bar{x})$ ?

Here we use Skolem's observation that given a first-order language $\mathscr{L}$, an $\mathscr{L}$-formula $\exists \bar{x} \phi(\bar{x})$ is satisfiable just in case the $\mathscr{L} \cup \{\bar{c}\}$-formula $\phi(\bar{c})$ is satisfiable for some fresh constants $\bar{c}$.

More generally, $\forall \bar{y} \exists \bar{x} \phi(\bar{y}, \bar{x})$ is satisfiable just in case $\forall \bar{y}\phi(f(\bar{y}))$ for some fresh function symbol $f$.

Thus we given our PNF formula $\phi = Q_{n} x_{n}\ldots Q_{1} x_{1} \psi$, we can start from the outside and work inward.  If we encounter a universal quantifier, leave it for now.  If we encounter an existential quantifier, say $\forall \bar{z} \exists{x_{j}} \psi_{j}$ we remove $\exists{x_{j}}$ and substitute into $\psi_{j}$ a fresh $f_j(\bar{z})$ for $x_j$.

When we have handled all the existenials, then we can simply remove the universal quantifiers.  

The result is a quantifier-free formula that is equisatisfiable to $\phi$.

You can experiment with Skolemization in our package:

https://github.com/aetilley/harrison-rust/blob/main/src/first_order_logic.rs#L1779

In [5]:
use harrison_rust::formula::Formula;
use harrison_rust::first_order_logic::Pred;

let formula_string = "exists x. forall w. exists y. (P(g(x, w)) ==>  ~R(y, x))";
let formula = Formula::<Pred>::parse(formula_string).unwrap();
let result = formula.skolemize();
result.pprint();

<<~P(g(c_x, w)) \/ ~R(f_y(w), c_x)>>


/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////


II **Herbrand**

A quantifier free formula of first-order logic is first order satisfiable just in case the set of all its *ground instances* is *propositionally satisfiable*.

We will not prove Herbrand's theorem, but we should define these italicized terms.

Roughly, a quantifier-free formula of first-order logic is said to be *propositionally satisfiable (valid)* if the formula of propositional logic that results from from substituting each first-order atomic subformula (e.g. $R(\bar{t})$) by a propositional logic sentential variable tagged with its name (say $p_{R(\bar{t})}$) is satisfiable (resp. valid).

We can then derive the following, which we state without proof:

**Theorem 1**:  A quantifier-free formula of first-order logic is propositionally valid iff it is first-ordervalid.

**Corollary 2**:  A quantifier-free formula of first-order logic *with no free variables* is propositionally satisfiable iff it is first-order satisfiable. 

We have implemented the DPLL algorithm

https://github.com/aetilley/harrison-rust/blob/main/src/formula.rs#L2213

to operate on a generic quantifier-free formulas (not just formulas of propositional logic).  So we can check for 
1) propositional satisfiability for quantifier-free first order formulas, and indeed
2) first-order satisfiability for quantifier-free first order formulas with no free variables.  E.g.

In [6]:
use harrison_rust::formula::Formula;
use harrison_rust::first_order_logic::Pred;

let formula_string = "R(c) /\\ ~R(d)";
let formula = Formula::<Pred>::parse(formula_string).unwrap();

println!("Is propositionally (and first-order) satisfiable?: {}", formula.dpll_sat());

Is propositionally (and first-order) satisfiable?: true


In [7]:

let formula_string = "R(x) /\\ ~R(y)";
let formula = Formula::<Pred>::parse(formula_string).unwrap();

println!("Is propositionally satisfiable?: {}", formula.dpll_sat());

Is propositionally satisfiable?: true


In [8]:
let formula_string = "R(x) /\\ ~R(x)";
let formula = Formula::<Pred>::parse(formula_string).unwrap();

println!("Is propositionally satisfiable?: {}", formula.dpll_sat());

Is propositionally satisfiable?: false


Unfortunately we do not have an analogous theorem to Theorem 1 above for satisfiability.  That is a quantifier-free formula of first-order logic may be propositionally satisfiable without being first-order satisfiable (if it has free variables).  

As a matter of fact the formula given above $R(x) \wedge ~R(y)$ is an example of this.  Clearly as a propositional logic formula this is equivalent to $p \wedge q$.  But to be first-order satisfiable, it must hold in some model $M$ under *all* valuations $v$ of the variables, and there are some valuations that map $x$ and $y$ to the same element.

And while our Skolemization transformation above is *satisfiability preserving*, it is not *validity preserving*, so we cannot simply test for the validity of a first-order formula by testing for the propositional validity of its Skolemization.

And remember that Skolemization introduces free variables at the last step when the universal quantifier's are dropped, so we can't check for a Skolemization's satisfiability by checking for its negation's validity.


_However_ Herbrand's theorem shows that we can get something almost as good.  We will need another definition...

Given a first-order formula $\phi$ involving with free variables $\bar{x}$ and involving constants / functions $\bar{c} , \bar{f}$, a *ground instance* of $\phi$ is the result of substituting in terms (called *ground terms*) built up out of $\bar{c}, \bar{f}$ (and no variables) for the free variables in $\phi$.  If there are no constants occuring in $\phi$ then we introdue a new constant $c$ so that there is at least one ground term.

For example, if $\phi$ were $R(x,y)$, then the only ground term would be $c$ and the only ground instance would be $R(c, c)$.

On the other hand if $\phi$ were $R(f(x),g(d))$, then the ground terms would be $$d, f(d), g(d), f(f(d), f(g(d)), g(f(d), g(g(d)), \ldots$$ and the ground instances of $\phi$ would be $$F(f(d), g(d)), F(f(f(d)), g(d)), F(f(g(d)), g(d)), \ldots$$

You can generate ground terms of any formula in the provided package with the function `ground_terms`.

https://github.com/aetilley/harrison-rust/blob/main/src/first_order_logic.rs#L1844

The following code computes all the ground terms of depth 2 over a constant $42$, a binary function $f$ and a unary function $g$.

In [9]:
use harrison_rust::formula::Formula;
use harrison_rust::first_order_logic::{Pred, Term};

use std::collections::{HashMap, HashSet};

let constants = HashSet::from([(String::from("42"), 0)]);
let functions = HashSet::from([(String::from("f"), 2), (String::from("g"), 1)]);
let empty_cache = HashMap::new();
let level = 2;
let result = Formula::ground_terms(&constants, &functions, level, &mut empty_cache.clone());
for term in result {
    term.pprint()
}


<<|f(g(42), 42)|>>
<<|g(f(42, 42))|>>
<<|f(42, f(42, 42))|>>
<<|f(42, g(42))|>>
<<|f(f(42, 42), 42)|>>
<<|g(g(42))|>>


()

Similarly, the ground *instances* correspond directly to length $n$ tuples of ground instances, where $n$ is the number of free variables in the case formula. We have the following code to compute such tuple.

https://github.com/aetilley/harrison-rust/blob/main/src/first_order_logic.rs#L1890

For instance, taking $\phi = R(f(x), y)$ the functions will be $f$ and the added constant $c$.  Tuples will be of length two.  The following call will compute all such tuples where exactly 3 (non-constant) function calls occur summing over all entries of the tuple:

In [10]:
use harrison_rust::formula::Formula;
use std::collections::{HashMap, HashSet};
use harrison_rust::first_order_logic::{Pred, Term};

let constants = HashSet::from([(String::from("c"), 0)]);
let functions = HashSet::from([(String::from("f"), 1)]);
let cache = HashMap::new();
let level = 3;
let size  = 2;

let result = Formula::ground_tuples(&constants, &functions, level, size, &mut cache.clone());

for tuple in result {
    for term in tuple {
        term.pprint();
    }
    println!()
}

<<|c|>>
<<|f(f(f(c)))|>>

<<|f(c)|>>
<<|f(f(c))|>>

<<|f(f(c))|>>
<<|f(c)|>>

<<|f(f(f(c)))|>>
<<|c|>>



()

Exciting huh?

At this point all the terms in Herbrand's Theorem should make sense:

**Theorem 3**:  A quantifier-free formula of first-order logic is first-order satisfiable just in case the set of all its ground instances is propositionally satisfiable

Now by the compactness theorem, this theorem is equivalent to

**Corollary 4**: A quantifier-free formula of first-order logic is first-order satisfiable just in case every finite subset of its ground instances is propositionally satisfiable, 

or equivalently

**Corollary 5**: A quantifier-free formula of first-order logic is first-order unsatisfiable just in case some finite subset of its ground instances is propositionally unsatisfiable, 




/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////


III **A semi-decision proceedure for validity**

With this, we are now in a position to design a semi-decision procedure for first-order unsatisfiability, because by the can enumerate larger and larger finite subsets of the set of all ground instances, testing each for satisfiability.  If the formula is unsatisfiable, we will eventually encompass this subset.  Otherwise we will run forever (or give up never knowing).

Indeed we will primarily use this as a semi-decision procedure for first order *validity*.   A first-order formula $\phi(\bar{x})$ with free variables $\bar{x}$ is valid just in case $\forall \bar{x} \phi(\bar{x}) $ (called its *generalization*) is valid, which (the latter having no free variables) is true just in case its negation is unsatisfiable.

So the outline of the semi-decision proceedure for validity will go as follows:

Given $\phi(\bar{x})$ with free variables $\bar{x}$:

1)  Compute $\psi := \lnot \forall\bar{x} \phi(\bar{x})$

2)  Compute the Skolemization $\chi$ of $\psi$ to get an equisatisfiable quantifier free formula.

3)  Iteratively check larger and larger subsets of all of $\chi$'s ground instances for unsatisfiability.

4)  If we find an unsatisfiable set (before giving up), we conclude that $\phi$ was valid.

Our library has implementation of more than one historical a function implement this check (each using different formula representations and different satisfiability checks).  We will look at the Davis-Putnam algorithm

https://github.com/aetilley/harrison-rust/blob/main/src/first_order_logic.rs#L2232

First let's check a valid formula:


In [3]:
use harrison_rust::formula::Formula;
use harrison_rust::first_order_logic::Pred;

let string = "exists x. exists y. (Loves(y, x) ==> Loves(y, best_friend(x)))";
let formula = Formula::<Pred>::parse(string).unwrap();
let compute_unsat_core = true;
let max_depth = 10;
Formula::davis_putnam(&formula, compute_unsat_core, max_depth)

Free variables are ["x", "y"]
Generating tuples for next level 0
Adding new formula to set: "<<Loves(c, c) /\\ ~Loves(c, best_friend(c))>>"
Generating tuples for next level 1
Adding new formula to set: "<<Loves(c, best_friend(c)) /\\ ~Loves(c, best_friend(best_friend(c)))>>"
Found 2 inconsistent ground instances of skolemized negation:
<<Loves(c, best_friend(c)) /\ ~Loves(c, best_friend(best_friend(c)))>>
<<Loves(c, c) /\ ~Loves(c, best_friend(c))>>
Formula is valid.


ValidityProved({[Fun("c", []), Fun("c", [])], [Fun("best_friend", [Fun("c", [])]), Fun("c", [])]})

Indeed the negation that we are checking for satisfiability is 
$$\forall x \forall y (\lnot Loves(y, best\_friend(x)) \wedge Loves (y, x))$$
which is clearly not satisfiable.

Let's look at a trickier one:

In [4]:
use harrison_rust::formula::Formula;
use harrison_rust::first_order_logic::Pred;

let string = "(forall x y. exists z. forall w. (P(x) /\\ Q(y) ==> R(z) /\\ U(w))) 
    ==> (exists x y. (P(x) /\\ Q(y))) ==> (exists z. R(z))";
let formula = Formula::<Pred>::parse(string).unwrap();
let compute_unsat_core = true;
let max_depth = 10;
Formula::davis_putnam(&formula, compute_unsat_core, max_depth)

Free variables are ["w", "x", "y"]
Generating tuples for next level 0
Adding new formula to set: "<<(((P(c_x) /\\ Q(c_y)) /\\ ((R(f_z(c_x, c_x)) \\/ ~P(c_x)) \\/ ~Q(c_x))) /\\ ((U(c_x) \\/ ~P(c_x)) \\/ ~Q(c_x))) /\\ ~R(c_x)>>"
Adding new formula to set: "<<(((P(c_x) /\\ Q(c_y)) /\\ ((R(f_z(c_x, c_y)) \\/ ~P(c_x)) \\/ ~Q(c_y))) /\\ ((U(c_x) \\/ ~P(c_x)) \\/ ~Q(c_y))) /\\ ~R(c_x)>>"
Adding new formula to set: "<<(((P(c_x) /\\ Q(c_y)) /\\ ((R(f_z(c_y, c_x)) \\/ ~P(c_y)) \\/ ~Q(c_x))) /\\ ((U(c_x) \\/ ~P(c_y)) \\/ ~Q(c_x))) /\\ ~R(c_y)>>"
Adding new formula to set: "<<(((P(c_x) /\\ Q(c_y)) /\\ ((R(f_z(c_y, c_y)) \\/ ~P(c_y)) \\/ ~Q(c_y))) /\\ ((U(c_x) \\/ ~P(c_y)) \\/ ~Q(c_y))) /\\ ~R(c_y)>>"
Adding new formula to set: "<<(((P(c_x) /\\ Q(c_y)) /\\ ((R(f_z(c_x, c_x)) \\/ ~P(c_x)) \\/ ~Q(c_x))) /\\ ((U(c_y) \\/ ~P(c_x)) \\/ ~Q(c_x))) /\\ ~R(c_x)>>"
Adding new formula to set: "<<(((P(c_x) /\\ Q(c_y)) /\\ ((R(f_z(c_x, c_y)) \\/ ~P(c_x)) \\/ ~Q(c_y))) /\\ ((U(c_y) \\/ ~P(c_x)) \\/ ~Q(c_y))) /\\

ValidityProved({[Fun("c_y", []), Fun("c_x", []), Fun("c_y", [])], [Fun("c_x", []), Fun("f_z", [Fun("c_x", []), Fun("c_y", [])]), Fun("c_x", [])]})

Next we check a formula similar to our first validity above but with universal quantifier:

In [5]:
let string = "forall x. exists y. (Loves(y, x) ==> Loves(y, best_friend(x)))";
let formula = Formula::<Pred>::parse(string).unwrap();
let compute_unsat_core = true;
let max_depth = 50;
Formula::davis_putnam(&formula, compute_unsat_core, max_depth)

Free variables are ["y"]
Generating tuples for next level 0
Adding new formula to set: "<<Loves(c_x, c_x) /\\ ~Loves(c_x, best_friend(c_x))>>"
Generating tuples for next level 1
Adding new formula to set: "<<Loves(best_friend(c_x), c_x) /\\ ~Loves(best_friend(c_x), best_friend(c_x))>>"
Generating tuples for next level 2
Adding new formula to set: "<<Loves(best_friend(best_friend(c_x)), c_x) /\\ ~Loves(best_friend(best_friend(c_x)), best_friend(c_x))>>"
Generating tuples for next level 3
Adding new formula to set: "<<Loves(best_friend(best_friend(best_friend(c_x))), c_x) /\\ ~Loves(best_friend(best_friend(best_friend(c_x))), best_friend(c_x))>>"
Generating tuples for next level 4
Adding new formula to set: "<<Loves(best_friend(best_friend(best_friend(best_friend(c_x)))), c_x) /\\ ~Loves(best_friend(best_friend(best_friend(best_friend(c_x)))), best_friend(c_x))>>"
Generating tuples for next level 5
Adding new formula to set: "<<Loves(best_friend(best_friend(best_friend(best_friend(best_f

BoundReached(50)

/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////


We can see that the negation of this formula:

$$\exists x \forall y (Loves(y, x) \wedge \lnot Loves(y, best\_friend(x)))$$

is obviously satisfiable in a world where there is one very popular person that everyone loves, but who has a friend that not everyone loves.

What about relational languages (ie. languages which no function symbols of positive arity)?  In this case the set of ground instances is finite, we we can hope to satisfy them all.

In [5]:
use harrison_rust::formula::Formula;
use harrison_rust::first_order_logic::Pred;

let string = "exists x y. (P(x) \\/ R(y))";
let formula = Formula::<Pred>::parse(string).unwrap();
let compute_unsat_core = true;
let max_depth = 10;
Formula::davis_putnam(&formula, compute_unsat_core, max_depth)

Generating tuples for next level 0
Adding new formula to set: "<<~P(c) /\\ ~R(c)>>"
Generating tuples for next level 1
Set of ground instances (of negation) was both finite and satisfiable.  Invalidity proved.


InvalidityProved

Note: You might think that you could also run a parallel process checking all models of the above language of increasing size looking for counterexamples. While this would indeed work for the example above, it does not work in general because there are first-order formulas which have counterexamples only in infinite models. For example consider the sentence expressing that every 1-1 function is onto:

$$\forall x, y (f(x) = f(y) \rightarrow x = y)  \rightarrow \forall w \exists z (f(z) = w)$$

This sentence is true in all finite models (remember our functions are always defined on the entire domain) but need not be true in infinite models.  

More generally, there is a very important theorem by Church says that no (full) decision procedure exists for first-order validity.